In [1]:
import requests, json, sys, folium
import pandas as pd
from postcodes_api import PostcodeApi

In [2]:
sch_stat = pd.read_excel('Datasets/scottish_schools_stats.xlsx')
sch_info = pd.read_excel('Datasets/scottish_schools_contact.xlsx', sheet_name='Open Schools')
dep = pd.read_excel('Datasets/postcode_deprivation.xlsx')

In [3]:
dep_rates = {}

for p,d in dep.values:
    dep_rates[p] = d

In [4]:
sch_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2461 entries, 0 to 2460
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Local Authority  2461 non-null   object
 1   SeedCode         2461 non-null   int64 
 2   School Name      2461 non-null   object
 3   School Type      2461 non-null   object
 4   Total pupils     2461 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 96.3+ KB


In [5]:
sch_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2458 entries, 0 to 2457
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Seed Code                                2458 non-null   int64 
 1   LA Name                                  2458 non-null   object
 2   Centre Type                              2458 non-null   object
 3   School Name                              2458 non-null   object
 4   Address Line1                            2458 non-null   object
 5   Address Line2                            2457 non-null   object
 6   Address Line3                            2458 non-null   object
 7   Post Code                                2458 non-null   object
 8   Unique Property Reference Number (UPRN)  2458 non-null   int64 
 9   Email                                    2458 non-null   object
 10  Phone Number                             2458 non-null   obj

In [6]:
sch_info = sch_info.rename(columns={'Seed Code' : 'SeedCode'})

In [7]:
sch_df = sch_info.merge(sch_stat, on='SeedCode', how='right')

In [8]:
sch_df.isna().sum()
sch_df.dropna(inplace=True)

In [9]:
sch_jsn = json.loads(sch_df.to_json(orient='records'))

In [ ]:
import folium
from folium.plugins import HeatMap

# Create a map centered at a specific location
m = folium.Map(location=[55.941457, -3.205744], zoom_start=12)  # London, UK

# Create a custom color scale from light to dark blue
colors = {
    1: '#08306b',  # Light blue
    2: '#08519c',
    3: '#3182bd',
    4: '#63b7f4',
    5: '#a6e1fa'   # Dark blue (most deprived)
}

l = len(sch_jsn)

# Create circles and digits for each data point
for sch in sch_jsn:
    l -= 1
    print(f'{l} remains')

    pos = sch['Post Code']

    try:
        loc = PostcodeApi().get_pos_info(pos)['loc']
    except TypeError:
        pass

    else:
        continue

    try:
        mag = dep_rates[pos] # magnitute
    except:
        mag = 3 # average
    
    folium.CircleMarker(
        location=loc,
        radius=25//mag,
        color=colors[mag],
        fill=True,
        fill_opacity=0.8,
    ).add_to(m)

    popup_html = f"""
    <h3>{sch['School Name_x']}</h3>
    <p><strong>Local Authority:</strong> {sch["LA Name"]}</p>
    <p><strong>Pupils:</strong> {sch["Total pupils"]}</p>
    <p><strong>Deprivation:</strong> {mag}</p>"""

    folium.Marker(
        location=loc,
        popup=folium.Popup(popup_html, max_width=150),
        icon=folium.DivIcon(html=f'<div style="width: 0px; height: 0px;"></div>'),
    ).add_to(m)

# Create a HeatMap layer using some sample data points
# heat_data = [[51.5074, -0.1278, 0.5], [51.5099, -0.1337, 0.2], [51.5113, -0.1219, 0.3]]
# HeatMap(heat_data).add_to(m)

# Create a custom HTML legend
legend_html = """
<div style="position: fixed; top: 10px; right: 10px; background-color: white; padding: 10px; border: 2px solid black; z-index: 1000;">
    <p><strong>Legend</strong></p>
    <p><span style="color: black;"><span style="background-color: #08306b; width: 20px; height: 20px; display: inline-block;"></span> 1 - Most Deprived</span></p>
    <p><span style="color: black;"><span style="background-color: #08519c; width: 20px; height: 20px; display: inline-block;"></span> 2</span></p>
    <p><span style="color: black;"><span style="background-color: #3182bd; width: 20px; height: 20px; display: inline-block;"></span> 3</span></p>
    <p><span style="color: black;"><span style="background-color: #63b7f4; width: 20px; height: 20px; display: inline-block;"></span> 4</span></p>
    <p><span style="color: black;"><span style="background-color: #a6e1fa; width: 20px; height: 20px; display: inline-block;"></span> 5 - Least Deprived</span></p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))


# Display the map (optional, this works in Jupyter Notebook)
m

In [11]:
l = len(sch_jsn)

for sch in sch_jsn:
    l -= 1
    print(l)
    pos = sch['Post Code']
    PostcodeApi().get_pos_info(pos)

2459
2458
2457
2456


KeyboardInterrupt: 

### Send bulk request

In [ ]:
import requests

# Define the URL
url = "https://api.postcodes.io/postcodes"

# Define the JSON payload
data = {
    "postcodes": ["PR3 0SG", "M45 6GN", "EX165BL","PR3 0SG", "M45 6GN", "EX165BL","PR3 0SG", "M45 6GN", "EX165BL","PR3 0SG", "M45 6GN", "EX165BL","PR3 0SG", "M45 6GN", "EX165BL","PR3 0SG", "M45 6GN", "EX165BL"]
}

# Set the headers
headers = {
    "Content-Type": "application/json"
}

# Send the POST request
response = requests.post(url, json=data, headers=headers)

# Check the response
if response.status_code == 200:
    # The request was successful, and you can parse the response JSON
    result = response.json()
    print(result)
else:
    # Handle the error
    print(f"Error: {response.status_code} - {response.text}")
